In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import pandas as pd
pd.options.display.max_columns=None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from src.client import ClientHelper
from src.analyse import OrdersAnalyser

In [ ]:
client_helper = ClientHelper()
orders = client_helper.get_all_orders()

In [ ]:
order_analyser = OrdersAnalyser(orders)
mean_price = order_analyser.calculate_mean_price()
display(mean_price)

In [ ]:
import plotly.express as px
orders = order_analyser.orders
fig = px.scatter(orders[orders['base_coin']=='ETH'], x='time', y="price",
                 size='executedQty')
fig.show()

In [ ]:
client = client_helper.client
prices = client.get_all_tickers()

In [ ]:
main_currency = 'USDT'
prices = pd.DataFrame(prices)
prices = prices[prices['symbol'].apply(lambda x: x[-len(main_currency):]==main_currency)]
prices['base_coin'] = prices['symbol'].apply(lambda x:x[:-len(main_currency)])
prices['quote_coin'] = main_currency
prices = prices.drop('symbol', axis=1)
eth_price = prices.loc[prices['base_coin']=='ETH', 'price'].item()
prices = pd.concat([prices, pd.DataFrame([{'price':eth_price, 'base_coin':'BETH', 'quote_coin':'USDT'}])])
prices['price'] = prices['price'].astype(float)
prices = pd.concat([prices, pd.DataFrame([{'price':1, 'base_coin':'USDT', 'quote_coin':'USDT'}])])

In [ ]:
history_assets = client_helper.get_history_assets('SPOT')
plot_df = history_assets.drop(['type', 'totalAssetOfBtc'], axis=1)
for c in ['RUB', 'USDT']:
    plot_df[c] = plot_df[c].apply(lambda x:x[:-2])

In [ ]:
import plotly.graph_objects as go
for c in labels:
    plot_df[c] = plot_df[c].astype(float)
labels = plot_df.columns[1:]
labels = [c for c in labels if c!='RUB']
values = plot_df[labels].iloc[-1]
for i, coin in enumerate(labels):
    price = prices.loc[prices['base_coin']==coin, 'price'].item()
    values[i]*=price
fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label',)])
fig.show()

In [ ]:
df = px.data.tips()
fig = px.pie(plot_df)
fig.show()

In [ ]:

# get market depth
depth = client.get_order_book(symbol='BNBBTC')

# place a test market buy order, to place an actual order use the create_order function
order = client.create_test_order(
    symbol='BNBBTC',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=100)

# get all symbol prices
prices = client.get_all_tickers()

# withdraw 100 ETH
# check docs for assumptions around withdrawals
from binance.exceptions import BinanceAPIException
try:
    result = client.withdraw(
        asset='ETH',
        address='<eth_address>',
        amount=100)
except BinanceAPIException as e:
    print(e)
else:
    print("Success")

# fetch list of withdrawals
withdraws = client.get_withdraw_history()

# fetch list of ETH withdrawals
eth_withdraws = client.get_withdraw_history(coin='ETH')

# get a deposit address for BTC
address = client.get_deposit_address(coin='BTC')

# get historical kline data from any date range

# fetch 1 minute klines for the last day up until now
klines = client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018")

# fetch weekly klines since it listed
klines = client.get_historical_klines("NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017")

# socket manager using threads
twm = ThreadedWebsocketManager()
twm.start()

# depth cache manager using threads
dcm = ThreadedDepthCacheManager()
dcm.start()

def handle_socket_message(msg):
    print(f"message type: {msg['e']}")
    print(msg)

def handle_dcm_message(depth_cache):
    print(f"symbol {depth_cache.symbol}")
    print("top 5 bids")
    print(depth_cache.get_bids()[:5])
    print("top 5 asks")
    print(depth_cache.get_asks()[:5])
    print("last update time {}".format(depth_cache.update_time))

twm.start_kline_socket(callback=handle_socket_message, symbol='BNBBTC')

dcm.start_depth_cache(callback=handle_dcm_message, symbol='ETHBTC')

# replace with a current options symbol
options_symbol = 'BTC-210430-36000-C'
dcm.start_options_depth_cache(callback=handle_dcm_message, symbol=options_symbol)

# join the threaded managers to the main thread
twm.join()
dcm.join()